In [1]:

from components.controller.bluetooth_controller import BlueToothCarControlSPP
from components.controller.bluetooth_SPP_server import start_bluetooth_server, start_bluetooth_server_v2
from components.gyroscope.gyroscope import AngularSpeedControlV2
from components.two_wheels import TwoWheelsV2
from components.camera import PicameraV2
#from components.controller.image_ml import ImageMLController
from data_collection.data_collection import LoggerSet, Logger
from multiprocessing import Manager

from components import default_loop, default_loop_v2, default_component_process_starter_v2, shared_value

import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)
from ctypes import c_double



%load_ext autoreload
%autoreload 2

In [2]:
logger_set = LoggerSet('../log/component_testing', overwrite_ok=True)
manager = Manager()





## Two Wheels

In [3]:
left_pwm_r, left_pwm_w = shared_value(manager, 'd', 0)
right_pwm_r, right_pwm_w = shared_value(manager, 'd', 0)

w_out, w_starter = default_component_process_starter_v2(
    TwoWheelsV2, 
    init_kwargs= dict(
        left_pin=33, 
        right_pin=32, 
        dir_pins=(16, 18), 
        logger_set=logger_set, 
        name='TwoWheelsV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
)
w = w_starter([left_pwm_r, right_pwm_r])

In [11]:
import time
left_pwm_w(100)
time.sleep(0.1)
left_pwm_w(0)

In [12]:
w.terminate()

# IMU

In [64]:
speed_r, speed_w = shared_value(manager, 'd', 0)
angular_velocity_r, angular_velocity_w = shared_value(manager, 'd', 0)

imu_out, imu_starter = default_component_process_starter_v2(
    AngularSpeedControlV2, 
    init_kwargs= dict(
        logger_set=logger_set, 
        name='AngularSpeedControlV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
)
imu = imu_starter([angular_velocity_r, speed_r])

In [75]:
speed_w(60)
angular_velocity_w(-1)

In [ ]:
imu_out[0](), imu_out[1]()

In [86]:
imu.terminate()

# Camera

In [97]:
cam_out, cam_starter = default_component_process_starter_v2(
    PicameraV2, 
    init_kwargs= dict(
        resolution=(114, 64), framerate=30,
        logger_set=logger_set, 
        name='PicameraV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
    shared_outputs_kwargs=dict(resolution=(114, 64))
)
cam = cam_starter()

[2:04:53.368415880] [27621]  INFO Camera camera_manager.cpp:297 libcamera v0.0.5+83-bde9b04f
[2:04:53.432958212] [27632]  INFO RPI vc4.cpp:437 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media0 and ISP device /dev/media2
[2:04:53.433228635] [27632]  INFO RPI pipeline_base.cpp:1101 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[2:04:53.563053977] [27621]  INFO Camera camera.cpp:1033 configuring streams: (0) 114x64-RGB888 (1) 1920x1080-SGRBG10_CSI2P
[2:04:53.563687045] [27632]  INFO RPI vc4.cpp:565 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 1920x1080-SGRBG10_1X10 - Selected unicam format: 1920x1080-pgAA


In [142]:
s = time.monotonic()
cam_out[0]()
e = time.monotonic()

e-s

0.0014964109996071784

In [144]:
cam.terminate()

In [146]:
cam.is_alive()

False

# Bluetooth

In [160]:
bt_ser_out, bt_ser = start_bluetooth_server_v2(manager)


Making discoverable...
Starting Serial Port Profile...
NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 92)


In [164]:
bt_ser_out[0]()

{'up': False}

In [159]:
bt_ser.terminate()

In [168]:

bt_out, bt_starter = default_component_process_starter_v2(
    BlueToothCarControlSPP, 
    init_kwargs= dict(
        logger_set=logger_set, 
        name='BlueToothCarControlSPP'
    ), 
    mainloop=default_loop_v2, 
    main_kwargs=dict(interval=1/30), 
    manager=manager, 
)


In [169]:
bt = bt_starter(bt_ser_out)

In [184]:
bt_out[0](),bt_out[1]()

(0, 0)

In [185]:
bt.terminate()

# ML Component


In [186]:
from components.controller.image_ml import ImageMLController
ml_out, ml_starter = default_component_process_starter_v2(
    ImageMLController, 
    init_kwargs= dict(
        #resolution=(114, 64), framerate=30,
        model_path='test.keras',
        logger_set=logger_set, 
        name='ImageMLController'
    ), 
    mainloop=default_loop_v2, 
    main_kwargs=dict(interval=1/60), 
    manager=manager, 
)


In [193]:
ml_out[0](),ml_out[1]()

(<tf.Tensor: shape=(), dtype=float32, numpy=1.1172317e-25>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.0058616e-26>)

In [189]:
ml = ml_starter(cam_out)

/home/kawa/projects/car1/.venv/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 30 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [194]:
ml.terminate()